# A Model of the Fed's View on Inflation

### Thomas Hasenzagl$^1$, Filippo Pellegrino$^2$, Lucrezia Reichlin$^3$, Giovanni Ricco$^4$

$^1$ University of Minnesota <br>
$^2$ London School of Economics and Political Science <br>
$^3$ London Business School, Now-Casting Economics, and CEPR <br>
$^4$ University of Warwick and OFCE-SciencesPo <br>


#### Notebook description

This notebook generates the charts from the in-sample estimation exercise from the paper "A Model of the Fed's View on Inflation". Before running this notebook you have to run `user_main.jl` using `run_type=1`. 


### Initialization: libraries and initial settings

In [ ]:
# ----- Load libraries -----

# Built-in
using Colors, CSV, Dates, DataFrames, Distributions, FileIO, JLD, ORCA, XLSX, KernelDensity, PlotlyJS;
pltjs = PlotlyJS

# Custom
include("./Metropolis-Within-Gibbs/MetropolisWithinGibbs.jl");
using Main.MetropolisWithinGibbs;

In [ ]:
# ----- Load results from output file -----

res           = load("$(pwd())/res_iis.jld");
nDraws        = res["nDraws"];
burnin        = res["burnin"];
σʸ            = res["σʸ"]';
date          = res["date"];
data          = res["data"] .* σʸ';
distr_α       = res["distr_α"];
chain_θ_bound = res["chain_θ_bound"]; # d, Z, R, c, T, Q, λ, ρ, total
MNEMONIC      = res["MNEMONIC"];

data[ismissing.(data)] .= NaN;

par_ind  = res["par_ind"];

par_size = SizeParSsm(sum(par_ind.d),
                                 sum(sum(par_ind.Z)),
                                 sum(sum(par_ind.R)),
                                 sum(par_ind.c),
                                 sum(sum(par_ind.T)),
                                 sum(sum(par_ind.Q)),
                                 sum(sum(par_ind.λ)),
                                 sum(sum(par_ind.ρ)),
                                 sum(par_ind.d) + sum(sum(par_ind.Z)) + sum(sum(par_ind.R)) +
                                    sum(par_ind.c) + sum(sum(par_ind.T)) + sum(sum(par_ind.Q)) +
                                    sum(sum(par_ind.λ)) + sum(sum(par_ind.ρ)));

# ----- Set titles and scales -----

titles = ["Real GDP", "Employment", "Unemployment rate", "Oil price", "CPI inflation", "Core CPI inflation", "UoM: Expected inflation", "SPF: Expected inflation"];
scales = ["Bil. Chn. 2009\$", "Thous.", "Percent", "\$/Barrel", "Percent", "Percent", "Percent", "Percent"];

In [ ]:
# ----- Get trends and cycles -----

n          = size(data)[2];
TT         = size(distr_α, 2);
ind_trends = [collect(9:3:size(distr_α)[1])[1:4]; collect(25:3:size(distr_α)[1])]; # There is no index for CPI inflation, please use if-else
ind_cycles = [collect(7:3:size(distr_α)[1])[1:4]; 19; 21; collect(23:3:size(distr_α)[1])];

Zᵖ = [ones(1, nDraws[2]-burnin[2]); 
      chain_θ_bound[1:7, burnin[2]+1:end]];

Zᵖ₂ = [zeros(1, nDraws[2]-burnin[2]); 
       chain_θ_bound[8:14, burnin[2]+1:end]];

Zᵖ₃ = [zeros(6, nDraws[2]-burnin[2]); 
       chain_θ_bound[15:16, burnin[2]+1:end]];

Zᵉ = [zeros(3, nDraws[2]-burnin[2]); 
      ones(1, nDraws[2]-burnin[2]); 
      chain_θ_bound[17:20, burnin[2]+1:end]];

Zᵉ₂ = [zeros(3, nDraws[2]-burnin[2]); 
       zeros(1, nDraws[2]-burnin[2]); 
       chain_θ_bound[21:24, burnin[2]+1:end]]

Zᵗ = [zeros(4, nDraws[2]-burnin[2]); 
      (1 ./ σʸ[end-3:end]) .* ones(4, nDraws[2]-burnin[2])];

In [ ]:
PC = zeros(n, TT, nDraws[2]-burnin[2]);
EP = zeros(n, TT, nDraws[2]-burnin[2]);
T  = zeros(n, TT, nDraws[2]-burnin[2]);

for i=1:nDraws[2]-burnin[2]    
    PC[:, :, i] = (Zᵖ[:, i] .* distr_α[1, :, i]') .+ 
                  (Zᵖ₂[:, i] .* distr_α[2, :, i]') .+
                  (Zᵖ₃[:, i] .* distr_α[3, :, i]');
    
    EP[:, :, i] = (Zᵉ[:, i] .* distr_α[4, :, i]') .+ 
                  (Zᵉ₂[:, i] .* distr_α[5, :, i]');

    T[:, :, i]  = Zᵗ[:, i] .* distr_α[6, :, i]';
end

In [ ]:
iC = distr_α[ind_cycles, :, :];
iT = distr_α[ind_trends, :, :];

PCᵐ = median(PC, dims=3)[:, :, 1]' .* σʸ';
EPᵐ = median(EP, dims=3)[:, :, 1]' .* σʸ';
Tᵐ  = median(T, dims=3)[:, :, 1]' .* σʸ';
iCᵐ = median(iC, dims=3)[:, :, 1]' .* σʸ';
iTᵐ = median(iT, dims=3)[:, :, 1]' .* [σʸ[1:4]' σʸ[7:8]'];

In [ ]:
PC95 = zeros(size(PCᵐ));
PC05 = zeros(size(PCᵐ));
PC84 = zeros(size(PCᵐ));
PC16 = zeros(size(PCᵐ));

EP95 = zeros(size(EPᵐ));
EP05 = zeros(size(EPᵐ));
EP84 = zeros(size(EPᵐ));
EP16 = zeros(size(EPᵐ));

T95 = zeros(size(Tᵐ));
T05 = zeros(size(Tᵐ));
T84 = zeros(size(Tᵐ));
T16 = zeros(size(Tᵐ));

iC95 = zeros(size(iCᵐ));
iC05 = zeros(size(iCᵐ));
iC84 = zeros(size(iCᵐ));
iC16 = zeros(size(iCᵐ));

iT95 = zeros(size(iTᵐ));
iT05 = zeros(size(iTᵐ));
iT84 = zeros(size(iTᵐ));
iT16 = zeros(size(iTᵐ));

for i=1:size(data, 2)
    for j=1:size(data, 1)
        
        PC95[j, i] = quantile(PC[i, j, :], 0.95) .* σʸ[i];
        PC05[j, i] = quantile(PC[i, j, :], 0.05) .* σʸ[i];
        PC84[j, i] = quantile(PC[i, j, :], 0.84) .* σʸ[i];
        PC16[j, i] = quantile(PC[i, j, :], 0.16) .* σʸ[i];
        
        EP95[j, i] = quantile(EP[i, j, :], 0.95) .* σʸ[i];
        EP05[j, i] = quantile(EP[i, j, :], 0.05) .* σʸ[i];
        EP84[j, i] = quantile(EP[i, j, :], 0.84) .* σʸ[i];
        EP16[j, i] = quantile(EP[i, j, :], 0.16) .* σʸ[i];

        T95[j, i] = quantile(T[i, j, :], 0.95) .* σʸ[i];
        T05[j, i] = quantile(T[i, j, :], 0.05) .* σʸ[i];
        T84[j, i] = quantile(T[i, j, :], 0.84) .* σʸ[i];
        T16[j, i] = quantile(T[i, j, :], 0.16) .* σʸ[i];
        
        iC95[j, i] = quantile(iC[i, j, :], 0.95) .* σʸ[i];
        iC05[j, i] = quantile(iC[i, j, :], 0.05) .* σʸ[i];
        iC84[j, i] = quantile(iC[i, j, :], 0.84) .* σʸ[i];
        iC16[j, i] = quantile(iC[i, j, :], 0.16) .* σʸ[i];
        

        if i < 5
            iT95[j, i] = quantile(iT[i, j, :], 0.95) .* σʸ[i];
            iT05[j, i] = quantile(iT[i, j, :], 0.05) .* σʸ[i];
            iT84[j, i] = quantile(iT[i, j, :], 0.84) .* σʸ[i];
            iT16[j, i] = quantile(iT[i, j, :], 0.16) .* σʸ[i];
        elseif i > 6
            iT95[j, i-2] = quantile(iT[i-2, j, :], 0.95) .* σʸ[i];
            iT05[j, i-2] = quantile(iT[i-2, j, :], 0.05) .* σʸ[i];
            iT84[j, i-2] = quantile(iT[i-2, j, :], 0.84) .* σʸ[i];
            iT16[j, i-2] = quantile(iT[i-2, j, :], 0.16) .* σʸ[i];
        end
    end
end

In [ ]:
# ----- Dates: add h dates to date -----

date=[date[i] for i=1:length(date)];
max_h = size(PCᵐ)[1] - size(date)[1];

for hz=1:max_h
    
    last_month = Dates.month(date[end]);
    last_year  = Dates.year(date[end]);
    new_month  = copy(last_month) + 3;
    new_year   = copy(last_year);
    
    if last_month + 1 > 12
        new_year  += 1;
        new_month  = 3;
    end
    
    new_entry = Dates.lastdayofquarter(Date(new_year, new_month, 1));
    date      = vcat(date, DateTime(new_entry));
end

In [ ]:
# Colour

c1 = "rgba(0, 48, 158, .75)"; #"rgba(0, 0, 158, .7)";
c2 = "rgba(255, 0, 0, .75)";
c3 = "rgba(255, 190, 0, .75)";

---

### Charts

#### Introduction

In [ ]:
figures   = Array{Any}(undef, 2);
ind_start = 49;

vv = [5, 5];

for i=1:2
    
    if i == 1
        trace1 = pltjs.bar(x=date, y=PCᵐ[1:end-max_h, vv[i]], name="Business cycle", marker_color=c1, showlegend=true, legendgroup="g1");
        trace2 = pltjs.bar(x=date, y=EPᵐ[1:end-max_h, vv[i]], name="Energy price cycle", marker_color=c2, showlegend=true, legendgroup="g2");
        trace3 = pltjs.bar(x=date, y=iCᵐ[1:end-max_h, vv[i]], name="Idiosyncratic cycle", marker_color=c3, showlegend=true, legendgroup="g3");
        trace4 = pltjs.scatter(x=date[1:end-max_h], y=PCᵐ[1:end-max_h, vv[i]]+EPᵐ[1:end-max_h, vv[i]]+iCᵐ[1:end-max_h, vv[i]], name="Cycle", mode="lines", line=attr(width=1.4, color="black", dash="dot"), showlegend=true, legendgroup="g4");
        
        databar = [trace1, trace2, trace3, trace4];    
        layout  = pltjs.Layout(title=string("Cycle: ", titles[vv[i]]), titlefont_size=10,
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", nticks=6, mirror=true, titlefont=attr(size=10), title=scales[vv[i]]),
                               barmode="relative", 
                               bargap=0,
                               bargroupgap=0);
        
        figures[i] = pltjs.plot(databar, layout);

    else
        trace0 = pltjs.scatter(x=date[1:end-max_h], y=data[1:end-max_h, vv[i]], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=true, legendgroup="g1");
        trace1 = pltjs.scatter(x=date[1:end-max_h], y=Tᵐ[1:end-max_h, vv[i]], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(x=date[1:end-max_h], y=T95[1:end-max_h, vv[i]], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=true, legendgroup="g4");
        trace3 = pltjs.scatter(x=date[1:end-max_h], y=T05[1:end-max_h, vv[i]], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(x=date[1:end-max_h], y=T84[1:end-max_h, vv[i]], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=true, legendgroup="g3");
        trace5 = pltjs.scatter(x=date[1:end-max_h], y=T16[1:end-max_h, vv[i]], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(x=date[1:end-max_h], y=Tᵐ[1:end-max_h, vv[i]], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=true, legendgroup="g2");

        layout  = pltjs.Layout(title=string("Trend: ", titles[vv[i]]), titlefont=attr(size=12),
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[vv[i]]));

        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6; trace0], layout);
    end
end

fig = [figures[1]; figures[2]];

# Bars
fig.plot.layout["barmode"]  = "relative";
fig.plot.layout["bargap"]   = 0.02;

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:2
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

fig.plot.layout["legend"] = attr(y=-0.1, x=0.176, font=attr(size=10), orientation="h")

savefig(fig, "./img/introduction.pdf", format="pdf")

#### Historical decomposition

In [ ]:
figures = Array{Any}(undef, 8);

for i=1:8
    trace1 = pltjs.bar(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, i], name="Business cycle", marker_color=c1, showlegend=i==1);
    trace2 = pltjs.bar(x=date[1:end-max_h], y=EPᵐ[1:end-max_h, i], name="Energy price cycle", marker_color=c2, showlegend=i==1);
    trace3 = pltjs.bar(x=date[1:end-max_h], y=iCᵐ[1:end-max_h, i], name="Idiosyncratic cycle", marker_color=c3, showlegend=i==1);
    trace4 = pltjs.scatter(x=date[1:end-max_h], y=PCᵐ[1:end-max_h, i]+EPᵐ[1:end-max_h, i]+iCᵐ[1:end-max_h, i], name="Total cycle", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==1)
    
    databar = [trace1, trace2, trace3, trace4];    
    layout  = pltjs.Layout(;title=titles[i], titlefont_size=10,
                           xaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90),
                           yaxis=attr(tickfont_size=14, showgrid=true, linecolor="black", mirror=true, titlefont=attr(size=14), title=scales[i]),
                           barmode="relative", 
                           bargap=0,
                           bargroupgap=0);

    figures[i] = pltjs.plot(databar, layout);
end

fig = [figures[1] figures[2]; figures[3] figures[4]; figures[5] figures[6]; figures[7] figures[8]];

# Bars
fig.plot.layout["barmode"] = "relative";
fig.plot.layout["bargap"]  = 0.02;

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:size(data, 2)
    fig.plot.layout["annotations"][i][:font][:size] = 20;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.1, x=0.075, font=attr(size=14))

savefig(fig, "./img/historical_decomposition.pdf", format="pdf")

#### Posteriors: key parameters

In [ ]:
# ----- Frequency -----

# - Chart
figures    = Array{Any}(undef, 2)
titles_sub = ["Frequency, BC cycle",
              "Frequency, EP cycle"];

for i=1:2
    
    # Frequency
    k             = kde(chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+i, :]);
    kde_x         = k.x;
    x_prior       = collect(0:0.01:pi);
    kde_prior     = pdf(Uniform(0, pi), x_prior);
    kde_posterior = k.density;
    
    
    trace1 = pltjs.scatter(x=x_prior, y=kde_prior, line=attr(width=1.4, color=c2), name="Prior", showlegend=i==1);
    trace2 = pltjs.scatter(x=kde_x, y=kde_posterior, line=attr(width=1.4, color=c1), name="Posterior", showlegend=i==1);

    layout = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, range=[0, pi]),
                           yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Density"));

    figures[i] = pltjs.plot([trace1; trace2], layout);
end


fig = [figures[1]; figures[2]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:2
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.1, x=0.375, font=attr(size=10));

In [ ]:
# ----- Persistence -----

# - Chart
figures2    = Array{Any}(undef, 2)
titles_sub = ["Persistence, BC cycle",
              "Persistence, EP cycle"];

for i=1:2
    
    # Persistence
    k             = kde(chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+par_size.λ+i, :]);
    kde_x         = k.x;
    x_prior       = collect(0:0.01:0.970);
    kde_prior     = pdf(Uniform(0, pi), x_prior);
    kde_posterior = k.density;
    
    
    trace1 = pltjs.scatter(x=x_prior, y=kde_prior, line=attr(width=1.4, color=c2), name="Prior", showlegend=false);
    trace2 = pltjs.scatter(x=kde_x, y=kde_posterior, line=attr(width=1.4, color=c1), name="Posterior", showlegend=false);

    layout = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, range=[0, 0.970]),
                           yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Density"));

    figures2[i] = pltjs.plot([trace1; trace2], layout);
end


fig = [figures2[1]; figures2[2]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:2
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.1, x=0.375, font=attr(size=10));

In [ ]:
# ----- Variance -----

# - Chart
figures_var    = Array{Any}(undef, 3)
titles_sub = ["Variance, BC cycle", 
              "Variance, EP cycle", 
              "Variance, Common trend"];

for i=1:3
    k             = kde(chain_θ_bound[par_size.R+par_size.d+par_size.Z+i, :]);
    kde_x         = k.x;
    x_prior       = collect(0:0.01:1.5);
    kde_prior     = pdf(InverseGamma(3, 1), x_prior);
    kde_posterior = k.density;

    trace1 = pltjs.scatter(x=x_prior, y=kde_prior, line=attr(width=1.4, color=c2), name="Prior", showlegend=false);
    trace2 = pltjs.scatter(x=kde_x, y=kde_posterior, line=attr(width=1.4, color=c1), name="Posterior", showlegend=false);

    layout = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, range=[0, 1.5]),
                           yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Density"));

    figures_var[i] = pltjs.plot([trace1; trace2], layout);
end


fig = [figures_var[1]; figures_var[2]; figures_var[3]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 800;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:3
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.06, x=0.375, font=attr(size=10));

In [ ]:
empty = pltjs.plot();
fig = [figures[1] figures[2] figures2[1] figures2[2]; figures_var[1] figures_var[2] figures_var[3] empty];

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:7
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.1, x=0.42, font=attr(size=10))

savefig(fig, "./img/posterior.pdf", format="pdf")

#### Loadings

In [ ]:
# ----- Loadings -----

# - Chart
figures_var = Array{Any}(undef, 4)
titles_sub = ["Coefficients for CPI inflation, Business cycle", 
                   "Coefficients for Core CPI inflation, Business cycle",
                   "Coefficients for CPI inflation, Energy price cycle", 
                   "Coefficients for Core CPI inflation, Energy price cycle"];

for i=1:4
    if i < 3
        k = kde(Zᵖ[5+i-1, :]);
    else
        k = kde(Zᵉ[5+i-3, :]);
    end
    
    kde_x         = k.x;
    x_prior       = collect(-5:0.1:5);
    kde_prior     = pdf(Normal(0, 1000), x_prior);
    kde_posterior = k.density;

    trace1 = pltjs.scatter(x=x_prior, y=kde_prior, line=attr(width=1.4, color=c2), name="Prior", showlegend=i==1);
    trace2 = pltjs.scatter(x=kde_x, y=kde_posterior, line=attr(width=1.4, color=c1), name="Posterior", showlegend=i==1);

    layout = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, range=[-2, 2]),
                           yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Density"));

    figures_var[i] = pltjs.plot([trace1; trace2], layout);
end


fig = [figures_var[1] figures_var[2]; figures_var[3] figures_var[4]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.05, x=0.4, font=attr(size=10))

savefig(fig, "./img/loadings.pdf", format="pdf")

#### Trends

In [ ]:
figures=Array{Any}(undef, 4)

cbo_gdp_potential = readtable("./data/potential.csv")[:output];
cbo_nairu = readtable("./data/potential.csv")[:nairu];
cbo_gdp_potential[ismissing.(cbo_gdp_potential)] .= NaN;
cbo_nairu[ismissing.(cbo_nairu)] .= NaN;

for i=1:4
    
    if i == 1
        trace_cbo = pltjs.scatter(x=date[1:end-max_h], y=cbo_gdp_potential, line=attr(width=1.4, color=c2), name="CBO", showlegend=true);
    elseif i == 3
        trace_cbo = pltjs.scatter(x=date[1:end-max_h], y=cbo_nairu, line=attr(width=1.4, color=c2), name="CBO", showlegend=false);
    end
    
    trace0 = pltjs.scatter(;x=date[1:end-max_h], y=data[1:end-max_h, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==4);
    trace1 = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
    trace2 = pltjs.scatter(;x=date[1:end-max_h], y=iT95[1:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==4);
    trace3 = pltjs.scatter(;x=date[1:end-max_h], y=iT05[1:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
    trace4 = pltjs.scatter(;x=date[1:end-max_h], y=iT84[1:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==4);
    trace5 = pltjs.scatter(;x=date[1:end-max_h], y=iT16[1:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
    trace6 = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==4);

     layout  = pltjs.Layout(;title=titles[i], titlefont=attr(size=12),
                            xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                            yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[i]));
    
     if i == 1 || i == 3
         figures[i] = pltjs.plot([trace_cbo; trace1; trace2; 
                                  trace1; trace3; 
                                  trace1; trace4;
                                  trace1; trace5; 
                                  trace6; trace0], layout);

     else
         figures[i] = pltjs.plot([trace1; trace2; 
                                  trace1; trace3; 
                                  trace1; trace4;
                                  trace1; trace5; 
                                  trace6; trace0], layout);
    end
end

fig = [figures[1] figures[2]; figures[3] figures[4]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.10, x=0.25, font=attr(size=10))

savefig(fig, "./img/trends_real.pdf", format="pdf")

In [ ]:
figures=Array{Any}(undef, 4)


for i=5:8
    
     trace0 = pltjs.scatter(;x=date[1:end-max_h], y=data[1:end-max_h, i], name="Data", mode="lines", line=attr(width=1.4, color="black"), showlegend=i==5);
     trace1 = pltjs.scatter(;x=date[1:end-max_h], y=Tᵐ[1:end-max_h, i], name="Trend", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
     trace2 = pltjs.scatter(;x=date[1:end-max_h], y=T95[1:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==5);
     trace3 = pltjs.scatter(;x=date[1:end-max_h], y=T05[1:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
     trace4 = pltjs.scatter(;x=date[1:end-max_h], y=T84[1:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==5);
     trace5 = pltjs.scatter(;x=date[1:end-max_h], y=T16[1:end-max_h, i], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
     trace6 = pltjs.scatter(;x=date[1:end-max_h], y=Tᵐ[1:end-max_h, i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==5);

     layout  = pltjs.Layout(;title=titles[i], titlefont=attr(size=12),
                            xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                            yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[i], range=[-2,8]));

     figures[i-4] = pltjs.plot([trace1; trace2; 
                              trace1; trace3; 
                              trace1; trace4;
                              trace1; trace5; 
                              trace6; trace0], layout);
end

fig = [figures[1]; figures[2]; figures[3]; figures[4]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.10, x=0.3, font=attr(size=10))

savefig(fig, "./img/trends_infl.pdf", format="pdf")

In [ ]:
long_term_exp = readtable("./data/INFCPI10YR.csv")[:INFCPI10YR];
long_term_exp[ismissing.(long_term_exp)] .= NaN;

i = 5;

trace0a = pltjs.scatter(;x=date[1:end-max_h], y=long_term_exp[1:size(date)[1]-max_h], name="SPF: 10y exp.", mode="lines", line=attr(width=1.4, color="red"), showlegend=true);
trace0b = pltjs.scatter(;x=date[1:end-max_h], y=data[1:end-max_h, i], name=titles[5], mode="lines", line=attr(width=1.4, color="black"), showlegend=true);
trace1  = pltjs.scatter(;x=date[1:end-max_h], y=Tᵐ[1:end-max_h, i], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
trace2  = pltjs.scatter(;x=date[1:end-max_h], y=T95[1:end-max_h, i], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=true);
trace3  = pltjs.scatter(;x=date[1:end-max_h], y=T05[1:end-max_h, i], name="", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
trace4  = pltjs.scatter(;x=date[1:end-max_h], y=T84[1:end-max_h, i], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=true);
trace5  = pltjs.scatter(;x=date[1:end-max_h], y=T16[1:end-max_h, i], name="", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
trace6  = pltjs.scatter(;x=date[1:end-max_h], y=Tᵐ[1:end-max_h, i], name="Trend", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=true);

layout  = pltjs.Layout(;title="Long-term expectations for CPI inflation", titlefont=attr(size=12),
                        xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                        yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title=scales[i]));

fig = [pltjs.plot([trace1; trace2; 
                       trace1; trace3; 
                       trace1; trace4;
                       trace1; trace5; 
                       trace6; trace0a; trace0b], layout)]


fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 300;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.22, x=0.185, font=attr(size=10))

savefig(fig, "./img/trends_infl_long_term_exp.pdf", format="pdf")

#### Parametric spectrum

In [ ]:
# ----- Spectral density distribution -----

num_s(ω, λ, ρ, σ²) = σ² .* (1 .+ ρ.^2 .- 2*ρ.*cos.(ω).*cos.(λ));
den_s(ω, λ, ρ)     = (2*pi) .* (1 .+ ρ.^4 .+ 4*(ρ^2).*(cos.(ω).^2) .- 4*ρ.*(1 .+ ρ.^2).*cos.(ω).*cos.(λ) .+ 2*(ρ.^2).*cos.(2*λ));
s(ω, λ, ρ, σ²)     = num_s(ω, λ, ρ, σ²)./den_s(ω, λ, ρ);

ω       = collect(0:0.01:pi);
s_distr = zeros(length(ω), 3, nDraws[2]-burnin[2]);

for i=1:3
    
    # parameters
    if i<3
        λ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+i, :];
        ρ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+par_size.λ+i, :];
        σ² = chain_θ_bound[par_size.R+par_size.d+par_size.Z+i, :];
    else
        λ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+2+4, :];
        ρ  = chain_θ_bound[par_size.R+par_size.d+par_size.Z+par_size.Q+par_size.c+par_size.T+par_size.λ+2+4, :];
        σ² = chain_θ_bound[par_size.R+par_size.d+par_size.Z+3+6+1, :];
    end
    
    # spectral density
    for draw=1:nDraws[2]-burnin[2]
        s_distr[:, i, draw] = s(ω[:], λ[draw], ρ[draw], σ²[draw]);
    end
end

# PC spectrum
PCᵐ_s  = median(s_distr[:, 1, :], dims=2);
PC95_s = zeros(size(PCᵐ_s));
PC05_s = zeros(size(PCᵐ_s));
PC84_s = zeros(size(PCᵐ_s));
PC16_s = zeros(size(PCᵐ_s));

EPᵐ_s  = median(s_distr[:, 2, :], dims=2);
EP95_s = zeros(size(EPᵐ_s));
EP05_s = zeros(size(EPᵐ_s));
EP84_s = zeros(size(EPᵐ_s));
EP16_s = zeros(size(EPᵐ_s));

iCᵐ_s  = median(s_distr[:, 3, :], dims=2);
iC95_s = zeros(size(iCᵐ_s));
iC05_s = zeros(size(iCᵐ_s));
iC84_s = zeros(size(iCᵐ_s));
iC16_s = zeros(size(iCᵐ_s));

for i=1:size(s_distr, 1)
    PC95_s[i] = quantile(s_distr[i, 1, :], 0.95);
    PC05_s[i] = quantile(s_distr[i, 1, :], 0.05);
    PC84_s[i] = quantile(s_distr[i, 1, :], 0.84);
    PC16_s[i] = quantile(s_distr[i, 1, :], 0.16);
    
    EP95_s[i] = quantile(s_distr[i, 2, :], 0.95);
    EP05_s[i] = quantile(s_distr[i, 2, :], 0.05);
    EP84_s[i] = quantile(s_distr[i, 2, :], 0.84);
    EP16_s[i] = quantile(s_distr[i, 2, :], 0.16);
    
    iC95_s[i] = quantile(s_distr[i, 1, :], 0.95);
    iC05_s[i] = quantile(s_distr[i, 1, :], 0.05);
    iC84_s[i] = quantile(s_distr[i, 1, :], 0.84);
    iC16_s[i] = quantile(s_distr[i, 1, :], 0.16);
end

In [ ]:
# ----- Chart -----

figures=Array{Any}(undef, 4)

titles_sub = ["Business cycle", "Energy price cycle", "Business cycle: spectral density", "Energy price cycle: spectral density"];

for i=1:4
    
    if i == 1 || i == 2
        layout  = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Percent"));
    else
        layout  = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false, titlefont=attr(size=10), title="Frequency"),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Spectral density"));
    end
    
    if i == 1
        trace1  = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 1], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2  = pltjs.scatter(;x=date[1:end-max_h], y=PC95[1:end-max_h, 1], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==1);
        trace3  = pltjs.scatter(;x=date[1:end-max_h], y=PC05[1:end-max_h, 1], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4  = pltjs.scatter(;x=date[1:end-max_h], y=PC84[1:end-max_h, 1], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==1);
        trace5  = pltjs.scatter(;x=date[1:end-max_h], y=PC16[1:end-max_h, 1], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6a = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 1], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==1);
        trace6b = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 1], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        trace7  = pltjs.scatter(;x=date[1:end-max_h], y=zeros(size(PCᵐ)[1]-max_h), name="", mode="lines", line=attr(width=1, color="black"), showlegend=false);
        figures[i] = pltjs.plot([trace6a; trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6b; trace7], layout);
    elseif i == 2
        trace1 = pltjs.scatter(;x=date[1:end-max_h], y=EPᵐ[1:end-max_h, 4], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[1:end-max_h], y=EP95[1:end-max_h, 4], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=date[1:end-max_h], y=EP05[1:end-max_h, 4], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[1:end-max_h], y=EP84[1:end-max_h, 4], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=date[1:end-max_h], y=EP16[1:end-max_h, 4], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[1:end-max_h], y=EPᵐ[1:end-max_h, 4], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        trace7 = pltjs.scatter(;x=date[1:end-max_h], y=zeros(size(EPᵐ)[1]-max_h), name="", mode="lines", line=attr(width=1, color="black"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6; trace7], layout);
    elseif i == 3
        trace1 = pltjs.scatter(;x=ω, y=PCᵐ_s[:], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=ω, y=PC95_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=ω, y=PC05_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=ω, y=PC84_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=ω, y=PC16_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=ω, y=PCᵐ_s[:], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6], layout);

    else
        trace1 = pltjs.scatter(;x=ω, y=EPᵐ_s[:], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=ω, y=EP95_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=ω, y=EP05_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=ω, y=EP84_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=ω, y=EP16_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=ω, y=EPᵐ_s[:], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6], layout);

    end
end


fig = [figures[1] figures[2]; figures[3] figures[4]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.165, x=0.33, font=attr(size=10), traceorder="normal")

savefig(fig, "./img/cycles_spectral.pdf", format="pdf")

In [ ]:
# ----- Chart -----

figures=Array{Any}(undef, 4)

titles_sub = ["Business cycle", "Energy price cycle", "Business cycle: spectral density", "Energy price cycle: spectral density"];

for i=1:4
    
    if i == 1 || i == 2
        layout  = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Percent"));
    else
        layout  = pltjs.Layout(;title=titles_sub[i], titlefont=attr(size=12),
                               xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false, titlefont=attr(size=10), title="Frequency"),
                               yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Spectral density"));
    end
    
    if i == 1
        trace1  = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 5], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2  = pltjs.scatter(;x=date[1:end-max_h], y=PC95[1:end-max_h, 5], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=i==1);
        trace3  = pltjs.scatter(;x=date[1:end-max_h], y=PC05[1:end-max_h, 5], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4  = pltjs.scatter(;x=date[1:end-max_h], y=PC84[1:end-max_h, 5], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=i==1);
        trace5  = pltjs.scatter(;x=date[1:end-max_h], y=PC16[1:end-max_h, 5], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6a = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 5], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=i==1);
        trace6b = pltjs.scatter(;x=date[1:end-max_h], y=PCᵐ[1:end-max_h, 5], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        trace7  = pltjs.scatter(;x=date[1:end-max_h], y=zeros(size(PCᵐ)[1]-max_h), name="", mode="lines", line=attr(width=1, color="black"), showlegend=false);
        figures[i] = pltjs.plot([trace6a; trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6b; trace7], layout);
    elseif i == 2
        trace1 = pltjs.scatter(;x=date[1:end-max_h], y=EPᵐ[1:end-max_h, 5], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=date[1:end-max_h], y=EP95[1:end-max_h, 5], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=date[1:end-max_h], y=EP05[1:end-max_h, 5], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=date[1:end-max_h], y=EP84[1:end-max_h, 5], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=date[1:end-max_h], y=EP16[1:end-max_h, 5], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=date[1:end-max_h], y=EPᵐ[1:end-max_h, 5], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        trace7 = pltjs.scatter(;x=date[1:end-max_h], y=zeros(size(EPᵐ)[1]-max_h), name="", mode="lines", line=attr(width=1, color="black"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6; trace7], layout);
    elseif i == 3
        trace1 = pltjs.scatter(;x=ω, y=PCᵐ_s[:], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=ω, y=PC95_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=ω, y=PC05_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=ω, y=PC84_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=ω, y=PC16_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=ω, y=PCᵐ_s[:], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6], layout);

    else
        trace1 = pltjs.scatter(;x=ω, y=EPᵐ_s[:], name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
        trace2 = pltjs.scatter(;x=ω, y=EP95_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace3 = pltjs.scatter(;x=ω, y=EP05_s[:], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
        trace4 = pltjs.scatter(;x=ω, y=EP84_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace5 = pltjs.scatter(;x=ω, y=EP16_s[:], name="CI, 84%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
        trace6 = pltjs.scatter(;x=ω, y=EPᵐ_s[:], name="Median", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"), showlegend=false);
        figures[i] = pltjs.plot([trace1; trace2; 
                                 trace1; trace3; 
                                 trace1; trace4;
                                 trace1; trace5; 
                                 trace6], layout);

    end
end


fig = [figures[1] figures[2]; figures[3] figures[4]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:4
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.165, x=0.33, font=attr(size=10), traceorder="normal")

savefig(fig, "./img/cycles_spectral_projection_infl.pdf", format="pdf")

#### Output gap

In [ ]:
i=1
iC = distr_α[ind_cycles, :, :];
iT = distr_α[ind_trends, :, :];
output_gap1 = PC[i,1:end-max_h,:] + EP[i,1:end-max_h,:] + iC[i,1:end-max_h,:];
output_gap1 = output_gap1 .* σʸ[i];
output_gap1 = 100*(output_gap1 ./ (iT[i,1:end-max_h,:] .* σʸ[i]));

OG95 = zeros(size(output_gap1,1));
OG05 = zeros(size(output_gap1,1));
OG50 = zeros(size(output_gap1,1));
OG84 = zeros(size(output_gap1,1));
OG16 = zeros(size(output_gap1,1));

for j=1:size(output_gap1, 1)  
    OG95[j, 1] = quantile(output_gap1[j,:], 0.95);
    OG05[j, 1] = quantile(output_gap1[j,:], 0.05);
    OG50[j, 1] = quantile(output_gap1[j,:], 0.50);
    OG84[j, 1] = quantile(output_gap1[j,:], 0.84); 
    OG16[j, 1] = quantile(output_gap1[j,:], 0.16);
end  

# ----- Output gap vs CBO -----

c1 = "rgba(0, 0, 158, .7)";
c2 = "rgba(255, 0, 0, .7)";
c3 = "rgba(0, 0, 0, .7)";

i=1;

gap_benchmark_file = CSV.read("./data/gap.csv") |> DataFrame;
cbo = gap_benchmark_file[!,:CBO] |> Array{Union{Missing, Float64}};
greenbook = gap_benchmark_file[!,:Greenbook] |> Array{Union{Missing, Float64}};
greenbook[ismissing.(greenbook)] .= NaN;


layout  = pltjs.Layout(;title="Output gap as a percentage of potential GDP", titlefont=attr(size=12),
                xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, nticks=20, tickangle=-90, zeroline=false),
                yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="Percent"));

i=1;

trace1 = pltjs.scatter(;x=date[1:end-max_h], y=OG50, name="", mode="lines", line=attr(color="rgba(0,0,0,0)"), showlegend=false);
trace2 = pltjs.scatter(;x=date[1:end-max_h], y=OG95, name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
trace3 = pltjs.scatter(;x=date[1:end-max_h], y=OG05, name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
trace4 = pltjs.scatter(;x=date[1:end-max_h], y=OG84, name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
trace5 = pltjs.scatter(;x=date[1:end-max_h], y=OG16, name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);
trace6 = pltjs.scatter(;x=date[1:end-max_h], y=OG50, name="Trend-Cycle Model", mode="lines", line=attr(width=1.4, color="rgba(0,0,255,1)"));
trace7 = pltjs.scatter(;x=date[1:end-max_h], y=zeros(size(OG50)[1]), name="", mode="lines", line=attr(width=1, color="black"), showlegend=false);
trace8 = pltjs.scatter(x=date[1:end-max_h], y=cbo, name="CBO", line=attr(width=1.4, color="rgb(255, 0, 0)"));
trace9 = pltjs.scatter(x=date[1:end-max_h], y=greenbook, name="Greenbook", line=attr(width=1.4, color="rgb(11, 102, 35)"));

fig = pltjs.plot([trace1; trace2; 
                        trace1; trace3; 
                        trace1; trace4;
                        trace1; trace5; 
                        trace6; trace7; trace8; trace9], layout);

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 300;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.25, x=0.3, font=attr(size=10))

savefig(fig, "./img/output_gap.pdf", format="pdf")

#### Oil trend

In [ ]:
figures = Array{Any}(undef, 1)

trace0aa = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, 4], name=titles[4], mode="lines", line=attr(width=1.4, color="black"), showlegend=true)
trace0a = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, 4], name=titles[4], mode="lines", line=attr(width=1.4, color="black"), showlegend=false)
trace1a  = pltjs.scatter(;x=date[1:end-max_h], y=iT95[1:end-max_h, 4], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=true);
trace2a  = pltjs.scatter(;x=date[1:end-max_h], y=iT05[1:end-max_h, 4], name="", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false);
trace3a  = pltjs.scatter(;x=date[1:end-max_h], y=iT84[1:end-max_h, 4], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=true);
trace4a  = pltjs.scatter(;x=date[1:end-max_h], y=iT16[1:end-max_h, 4], name="", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false);

trace0bb = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, 5], name=titles[7], mode="lines", line=attr(width=1.4, color=c1), showlegend=true, yaxis="y2");
trace0b = pltjs.scatter(;x=date[1:end-max_h], y=iTᵐ[1:end-max_h, 5], name=titles[7], mode="lines", line=attr(width=1.4, color=c1), showlegend=false, yaxis="y2");
trace1b  = pltjs.scatter(;x=date[1:end-max_h], y=iT95[1:end-max_h, 5], name="CI, 90%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false, yaxis="y2");
trace2b  = pltjs.scatter(;x=date[1:end-max_h], y=iT05[1:end-max_h, 5], name="", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .40)", line_color="transparent", showlegend=false, yaxis="y2");
trace3b  = pltjs.scatter(;x=date[1:end-max_h], y=iT84[1:end-max_h, 5], name="CI, 68%", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false, yaxis="y2");
trace4b  = pltjs.scatter(;x=date[1:end-max_h], y=iT16[1:end-max_h, 5], name="", mode="lines", fill="tonexty", fillcolor="rgba(185, 185, 185, .70)", line_color="transparent", showlegend=false, yaxis="y2");

layout = pltjs.Layout(;title=string("Idiosyncratic trends"), titlefont=attr(size=14), width=800, height=300,
        xaxis=attr(tickfont_size=10, nticks=20, tickangle=-90, showgrid=true, linecolor="black", mirror=true, zeroline=false),
        yaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, titlefont=attr(size=10), title="\$/Barrel"),
        yaxis2=attr(titlefont=attr(size=10), zeroline=false, tickfont_size=10, showgrid=false, title="Percent", titleangle=90, overlaying="y", side="right"));

fig = pltjs.plot([trace0aa; trace1a;
                  trace0a; trace2a;
                  trace0a; trace3a;
                  trace0a; trace4a; 
                  trace0bb; trace1b;
                  trace0b; trace2b;
                  trace0b; trace3b;
                  trace0b; trace4b], layout)

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Legend
fig.plot.layout["legend"] = attr(orientation="h", y=-0.2, x=0.21, font=attr(size=10))

savefig(fig, "./img/trends_idio_oil.pdf", format="pdf")

#### Slope

In [ ]:
function standardize(x)
    return x.-mean(x)
end    

yy = standardize(PCᵐ[1:end-max_h, 5]);
xx = standardize(PCᵐ[1:end-max_h, 3]);
b1 = (xx'*xx)\xx'*yy;
z  = xx*b1;

yy2 = standardize(data[1:end-max_h, 5]);
xx2 = standardize(data[1:end-max_h, 3]);
b2  = (xx2'*xx2)\xx2'*yy2;
z2  = xx2*b2;

figures = Array{Any}(undef, 1)
trace1a = pltjs.scatter(;x=xx[1:end-18], y=yy[1:end-18], mode="markers", marker=attr(color="red"), opacity=0.45, showlegend=false);
trace1b = pltjs.scatter(;x=xx[end-17:end], y=yy[end-17:end], mode="markers", marker=attr(color="red", symbol="x"), opacity=1, showlegend=false);
trace2 = pltjs.scatter(;x=xx, y=z, showlegend=false, line=attr(color="darkred"));
trace3 = pltjs.scatter(;x=xx2, y=yy2, mode="markers", marker=attr(color="blue"), opacity=0.45, showlegend=false);
trace4 = pltjs.scatter(;x=xx2, y=z2, showlegend=false, line=attr(color="blue"));

# Trace1b includes data from 2014 to 2018

layout = pltjs.Layout(;titlefont=attr(size=14), width=800, height=300,
        xaxis=attr(tickfont_size=10, showgrid=true, title="Unemployment Rate"),
        yaxis=attr(tickfont_size=10, showgrid=true, title="Headline Inflation"))

fig = pltjs.plot([trace1a; trace1b; trace2; trace3; trace4], layout)

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 800;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

savefig(fig, "./img/slope_cpi.pdf", format="pdf")

#### Regression output for global demand data

In [ ]:
# Data

# Dependent variables
PC_delta = diff(PCᵐ[1:end-max_h-2,1]);
EP_delta = diff(EPᵐ[1:end-max_h-2,4]);
y_delta = [PC_delta EP_delta];

# Cut dependent data to match the predictors
y_delta = y_delta[5:end,:];

# Predictors
global_demand = Array{Float64,2}(readtable("./data/global.csv")[:,[:IP, :BDI, :GCI]]);
global_demand[:,1] = [NaN; diff(log.(global_demand[:,1]))];
global_demand[:,2] = [NaN; diff(log.(global_demand[:,2]))];
global_demand = global_demand[2:end,:];

# Titles axes
y_title=["BC", "EP"];
x_title=["IP", "BDI", "GCI"];

# Subplots

figures=Array{Any}(undef, 6)

for i=1:2
    for j=1:3        
        
        # Raw data for regression
        yy=y_delta[:,i];
        xx=global_demand[:,j];
        
        # De-mean
        yy = standardize(yy);
        xx = standardize(xx);
        
        # Run regression
        b1 = (xx'*xx)\xx'*yy;
        z = xx*b1;
        
        # Make subplot
        trace1 = pltjs.scatter(;x=xx, y=yy, mode="markers", marker=attr(color="blue"), opacity=0.5, showlegend=false);
        trace2 = pltjs.scatter(;x=xx, y=z, showlegend=false, line=attr(color="black"));
        
        layout = pltjs.Layout(;titlefont=attr(size=14), width=800, height=300,
                 xaxis=attr(tickfont_size=10, showgrid=true, title=x_title[j]),
                 yaxis=attr(tickfont_size=10, showgrid=true, title=y_title[i]))
        
        # Store subplot
        figures[j+(i-1)*3] = pltjs.plot([trace1; trace2], layout);
    end
end

In [ ]:
fig = [figures[1] figures[2] figures[3]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 300;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

savefig(fig, "./img/global_exploratory_BC.pdf", format="pdf")

In [ ]:
fig = [figures[4] figures[5] figures[6]];

fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 300;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

savefig(fig, "./img/global_exploratory_EP.pdf", format="pdf")

##### Save key output to csv

In [ ]:
OG=median((PC[1,:,:]+EP[1,:,:]+iC[1,:,:])./iT[1,:,:], dims=2)[:]*100;
CSV.write("./csv_output/gap.csv", DataFrame(timestamp=date, x=OG))
CSV.write("./csv_output/PC.csv", DataFrame(timestamp=date, x=PCᵐ[:,1]))
CSV.write("./csv_output/EP.csv", DataFrame(timestamp=date, x=EPᵐ[:,4]))

##### Appendix

#### Coefficients: loadings

In [ ]:
c1 = "rgba(0, 48, 158, .75)";

figures=Array{Any}(undef, 24)

titles_coeff = ["Coefficient for Employment, BC Cycle",
                "Coefficient for URATE, BC Cycle",
                "Coefficient for Oil, BC Cycle",
                "Coefficient for Inflation, BC Cycle",
                "Coefficient for Core Inflation, BC Cycle",
                "Coefficient for UoM, BC Cycle",
                "Coefficient for SPF, BC Cycle",
                "Coefficient for Employment, aux BC Cycle",
                "Coefficient for URATE, aux BC Cycle",
                "Coefficient for Oil, aux BC Cycle",
                "Coefficient for Inflation, aux BC Cycle",
                "Coefficient for Core Inflation, aux BC Cycle",
                "Coefficient for UoM, aux BC Cycle",
                "Coefficient for SPF, aux BC Cycle",
                "Coefficient for UoM, aux^2 BC Cycle",
                "Coefficient for SPF, aux^2 BC Cycle",
                "Coefficient for Inflation, EP Cycle",
                "Coefficient for Core Inflation, EP Cycle",   
                "Coefficient for UoM, EP Cycle",
                "Coefficient for SPF, EP Cycle",
                "Coefficient for Inflation, aux EP Cycle",
                "Coefficient for Core Inflation, aux EP Cycle",   
                "Coefficient for UoM, aux EP Cycle",
                "Coefficient for SPF, aux EP Cycle"];

for i=1:24
    trace1 = pltjs.histogram(x=chain_θ_bound[i,nDraws[2]-burnin[2]+1:nDraws[2]], opacity=0.75, showlegend=false, histnorm="probability", nbinsx=45);
    layout = pltjs.Layout(;title=titles_coeff[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, range =[-3, 3]),
                           yaxis=attr(tickfont_size=10, nticks=10, tickmode="linear", tick0=0, dtick=0.04, showgrid=true, range=[0,0.16], linecolor="black", mirror=true, titlefont=attr(size=10), title="Relative frequency"), range=[0, 0.14]);

    figures[i] = pltjs.plot(trace1, layout);
end

fig = [figures[1] figures[2] figures[3] figures[4]; figures[5] figures[6] figures[7] figures[8];
       figures[9] figures[10] figures[11] figures[12]; figures[13] figures[14] figures[15] figures[16];
       figures[17] figures[18] figures[19] figures[20]; figures[21] figures[22] figures[23] figures[24]]

# Size
fig.plot.layout["width"]  = 1200;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:24
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

savefig(fig, "./img/appendix_Z_params.pdf", format="pdf")

#### Coefficients: drifts

In [ ]:
start = sum(par_ind.d) + sum(par_ind.Z) + sum(par_ind.R);

figures=Array{Any}(undef, 2)

titles_coeff = ["Drift of idio trend, GDP"; "Drift of idio trend, Employment"]  

for i=1:2
    trace1 = pltjs.histogram(x=chain_θ_bound[start+i,nDraws[2]-burnin[2]+1:nDraws[2]], opacity=0.75, showlegend=false, histnorm="probability", nbinsx=45);
    layout = pltjs.Layout(;title=titles_coeff[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true),
                           yaxis=attr(tickfont_size=10, nticks=10, tickmode="linear", tick0=0, dtick=0.02, showgrid=true, linecolor="black", mirror=true, titlefont=attr(size=10), title="Relative frequency"));

    figures[i] = pltjs.plot(trace1, layout);
end

fig = [figures[1]; figures[2]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 400;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:2
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

savefig(fig, "./img/appendix_c_params.pdf", format="pdf")

#### Coefficients: shock variance

In [ ]:
start = sum(par_ind.d) + sum(par_ind.Z) + sum(par_ind.R) + sum(par_ind.c);

figures=Array{Any}(undef, 18)

titles_coeff = ["Shock variance for BC Cycle",
                "Shock variance for EP Cycle",
                "Shock variance for Common Trend",
                "Shock variance for idio cycle, GDP",
                "Shock variance for idio trend, GDP",  
                "Shock variance for idio cycle, Employment",
                "Shock variance for idio trend, Employment",  
                "Shock variance for idio cycle, URATE",
                "Shock variance for idio trend, URATE",
                "Shock variance for idio cycle, Oil price",
                "Shock variance for idio trend, Oil price",     
                "Shock variance for idio cycle, Inflation",
                "Shock variance for idio cycle, Core inflation",
                "Shock variance for idio cycle, SPF",
                "Shock variance for idio trend, SPF",
                "Shock variance for idio cycle, UoM",
                "Shock variance for idio trend, UoM"];

for i=1:17
    trace1 = pltjs.histogram(x=chain_θ_bound[start+i,nDraws[2]-burnin[2]+1:nDraws[2]], opacity=0.75, showlegend=false, histnorm="probability", nbinsx=45);
    layout = pltjs.Layout(;title=titles_coeff[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true),
                           yaxis=attr(tickfont_size=10, nticks=10, tickmode="linear", tick0=0, dtick=0.04, showgrid=true, range=[0, 0.14], linecolor="black", mirror=true, titlefont=attr(size=10), title="Relative frequency"));

    figures[i] = pltjs.plot(trace1, layout);
end
figures[18] = pltjs.plot();

fig = [figures[1] figures[2] figures[3]; figures[4] figures[5] figures[6]; figures[7] figures[8] figures[9]; figures[10] figures[11] figures[12]; figures[13] figures[14] figures[15]; figures[16] figures[17] figures[18]];

# Size
fig.plot.layout["width"]  = 1000;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:17
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

savefig(fig, "./img/appendix_Q_params.pdf", format="pdf")

#### Coefficients: frequency

In [ ]:
start = par_size.d+par_size.Z+par_size.R+par_size.c+par_size.T+par_size.Q;

figures=Array{Any}(undef, 10)

titles_coeff = ["Frequency of BC Cycle",
                "Frequency of EP Cycle",
                "Frequency of idio Cycle, GDP",
                "Frequency of idio Cycle, Employment",
                "Frequency of idio Cycle, URATE Rate",
                "Frequency of idio Cycle, Oil price",
                "Frequency of idio Cycle, Inflation",
                "Frequency of idio Cycle, Core inflation",
                "Frequency of idio Cycle, UoM",
                "Frequency of idio Cycle, SPF"]  

for i=1:10
    trace1 = pltjs.histogram(x=chain_θ_bound[start+i,nDraws[2]-burnin[2]+1:nDraws[2]], opacity=0.75, showlegend=false, histnorm="probability", nbinsx=45);
    layout = pltjs.Layout(;title=titles_coeff[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, range=[0,3.14]),
                           yaxis=attr(tickfont_size=10, nticks=10, tickmode="linear", tick0=0, dtick=0.04, showgrid=true, range=[0,0.16], linecolor="black", mirror=true, titlefont=attr(size=10), title="Relative frequency"));

    figures[i] = pltjs.plot(trace1, layout);
end

fig = [figures[1] figures[2]; figures[3] figures[4]; figures[5] figures[6]; figures[7] figures[8]; figures[9] figures[10]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:10
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

savefig(fig, "./img/appendix_lambda_params.pdf", format="pdf")

In [ ]:
start = par_size.d+par_size.Z+par_size.R+par_size.c+par_size.T+par_size.Q + par_size.λ;

figures=Array{Any}(undef, 10)

titles_coeff = ["Persistence of BC Cycle",
                "Persistence of EP Cycle",
                "Persistence of idio Cycle, GDP",
                "Persistence of idio Cycle, Employment",
                "Persistence of idio Cycle, URATE Rate",
                "Persistence of idio Cycle, Oil price",
                "Persistence of idio Cycle, Inflation",
                "Persistence of idio Cycle, Core inflation",
                "Persistence of idio Cycle, UoM",
                "Persistence of idio Cycle, SPF"]  

for i=1:10
    trace1 = pltjs.histogram(x=chain_θ_bound[start+i,nDraws[2]-burnin[2]+1:nDraws[2]], opacity=0.75, showlegend=false, histnorm="probability", nbinsx=45);
    layout = pltjs.Layout(;title=titles_coeff[i], titlefont=attr(size=12),
                           xaxis=attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, range=[0,1]),
                           yaxis=attr(tickfont_size=10, nticks=10, tickmode="linear", tick0=0, dtick=0.04, showgrid=true, range=[0,0.16], linecolor="black", mirror=true, titlefont=attr(size=10), title="Relative frequency"));

    figures[i] = pltjs.plot(trace1, layout);
end

fig = [figures[1] figures[2]; figures[3] figures[4]; figures[5] figures[6]; figures[7] figures[8]; figures[9] figures[10]];

# Size
fig.plot.layout["width"]  = 800;
fig.plot.layout["height"] = 600;

# Bars
fig.plot.layout["bargap"]      = 0;
fig.plot.layout["bargroupgap"] = 0;

# Margins
fig.plot.layout["margin"][:b]  = 40;
fig.plot.layout["margin"][:t]  = 40;
fig.plot.layout["margin"][:r]  = 40;
fig.plot.layout["margin"][:l]  = 40;

# Title size
for i=1:10
    fig.plot.layout["annotations"][i][:font][:size] = 12;
end

savefig(fig, "./img/appendix_rho_params.pdf", format="pdf")